####  Google-bert-multilingual pretrained-BERT

In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [2]:
# Load pre-trained model tokenizer (vocabulary)
modelname = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(modelname)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [3]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained(modelname)
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
  

In [4]:
tokenizer = BertTokenizer.from_pretrained(modelname)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
text = "안녕하세요. SK텔레콤에 가서 핸드폰을 개통할까요?"
target = '##통'
tokenized_text = tokenizer.tokenize(text)

In [6]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = tokenized_text.index(target)
tokenized_text[masked_index] = '[MASK]'

In [7]:
print(tokenized_text)

['안', '##녕', '##하', '##세', '##요', '.', 'SK', '##텔', '##레', '##콤', '##에', '가', '##서', '핸', '##드', '##폰', '##을', '개', '[MASK]', '##할', '##까', '##요', '?']


In [8]:
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

[9521, 118741, 35506, 24982, 48549, 119, 21275, 119354, 56645, 119309, 10530, 8843, 12424, 9962, 15001, 119401, 10622, 8857, 103, 14843, 118671, 48549, 136]


In [9]:
def eos_index(tokenized_text):
    end_marks = ['.', '?', '!']
    idx_list=[]
    for item in end_marks:        
        try:
            idx_list.append(tokenized_text.index(item))
        except:
            pass
    return min(idx_list)

In [10]:
eos_idx = eos_index(tokenized_text)

# Define sentence A and B indices associated to 1st and 2nd sentences
segments_ids = [1] * len(tokenized_text)
# this is for the first sentence. 
for i in range(eos_idx):
    segments_ids[i] =0 

In [11]:
mask_ids = [1] * len(tokenized_text)
mask_ids[masked_index] = 0

In [12]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
masks_tensors = torch.tensor([mask_ids])
segments_tensors = torch.tensor([segments_ids])

In [13]:
# Predict all tokens
predictions = model(tokens_tensor, segments_tensors)
predicted_index = torch.argmax(predictions[0, masked_index]).item() # argmax로
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])

In [14]:
print("Original:", text)
print("Masked:", " ".join(tokenized_text))

Original: 안녕하세요. SK텔레콤에 가서 핸드폰을 개통할까요?
Masked: 안 ##녕 ##하 ##세 ##요 . SK ##텔 ##레 ##콤 ##에 가 ##서 핸 ##드 ##폰 ##을 개 [MASK] ##할 ##까 ##요 ?


In [15]:
print("Predicted token:", predicted_token)

Predicted token: ['##량']


In [16]:
predictions.shape # reconstruct 된 문장이므로 22개의 모든 token에 대한 probability를 출력

torch.Size([1, 23, 119547])

In [17]:
prob_masked_token = predictions[0, masked_index]
prob_masked_token.shape

torch.Size([119547])

In [18]:
values, indices = torch.topk(prob_masked_token, 10)

print(values)
print(indices)

tensor([15.6674, 15.1089, 14.6751, 14.3621, 13.9021, 13.8833, 13.7950, 13.7224,
        13.4476, 13.3646], grad_fn=<TopkBackward>)
tensor([ 44321,  12030,  20626,  61844,  58303, 119259,  18227,  42337, 118739,
         21789])


In [19]:
predicted_token = tokenizer.convert_ids_to_tokens(indices.numpy())
print(predicted_token)

['##량', '##인', '##조', '##별', '##입', '##척', '##화', '##방', '##념', '##개']


In [20]:
def generate_topk_words(test_str):
    text = test_str
    tokenized_text = tokenizer.tokenize(text)
    
    masked_index = tokenized_text.index('_')
    tokenized_text[masked_index] = '[MASK]'
    
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Define sentence A and B indices associated to 1st and 2nd sentences
    eos_idx = eos_index(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    # this is for the first sentence 'hello'. 
    for i in range(eos_idx):
        segments_ids[i] =0 

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    # Predict all tokens
    predictions = model(tokens_tensor, segments_tensors)
    prob_masked_token = predictions[0, masked_index]

    print("Original:", text)
    print("Masked:", " ".join(tokenized_text))
    
    values, indices = torch.topk(prob_masked_token, 10)
    predicted_token = tokenizer.convert_ids_to_tokens(indices.numpy())
    print('Predicted top-k tokens: ',  predicted_token)

In [21]:
generate_topk_words('안녕하세요. 오늘 아침에 아주 맛있는 _을 먹었습니다.')

Original: 안녕하세요. 오늘 아침에 아주 맛있는 _을 먹었습니다.
Masked: 안 ##녕 ##하 ##세 ##요 . 오 ##늘 아 ##침 ##에 아 ##주 맛 ##있는 [MASK] 을 먹 ##었 ##습 ##니다 .
Predicted top-k tokens:  ['밥', '음', '[UNK]', '것', '맛', '점', '물', '겁', '음악', '빵']


In [22]:
generate_topk_words('만나서 반갑습니다. 오늘 아침에 재미있는 _을 읽었습니다.')

Original: 만나서 반갑습니다. 오늘 아침에 재미있는 _을 읽었습니다.
Masked: 만 ##나 ##서 반 ##갑 ##습 ##니다 . 오 ##늘 아 ##침 ##에 재 ##미 ##있는 [MASK] 을 읽 ##었 ##습 ##니다 .
Predicted top-k tokens:  ['책', '글', '것', '[UNK]', '역', '점', '음악', '집', '말', '문']


In [23]:
generate_topk_words('반가워요! 오늘 아침에 신나는 _를 들었습니다.')

Original: 반가워요! 오늘 아침에 신나는 _를 들었습니다.
Masked: 반 ##가 ##워 ##요 ! 오 ##늘 아 ##침 ##에 신 ##나는 [MASK] 를 들 ##었 ##습 ##니다 .
Predicted top-k tokens:  ['노래', '"', "'", '(', ')', '이야기', '!', '곡', '[UNK]', ',']


In [24]:
generate_topk_words('안녕하세요? 저는 오늘 아_에 신나는 음악을 들었습니다.')

Original: 안녕하세요? 저는 오늘 아_에 신나는 음악을 들었습니다.
Masked: 안 ##녕 ##하 ##세 ##요 ? 저 ##는 오 ##늘 아 [MASK] 에 신 ##나는 음악 ##을 들 ##었 ##습 ##니다 .
Predicted top-k tokens:  ['##프리카', '##침', '##시아', '.', '##이', '##역', '##래', '##리', '##날', '##늘']


In [25]:
generate_topk_words('또 만났네요! 저는 _ 할 때 이 세상에서 제일 행복 합니다.')

Original: 또 만났네요! 저는 _ 할 때 이 세상에서 제일 행복 합니다.
Masked: 또 만 ##났 ##네 ##요 ! 저 ##는 [MASK] 할 때 이 세 ##상 ##에서 제 ##일 행 ##복 합 ##니다 .
Predicted top-k tokens:  ['나', '[UNK]', '노래', '이', '행', '말', '복', '다', '모두', '함께']


###  Assign a score to a sentence
- Evaluate whether a sentence is natural based on the language model.

In [26]:
import math

In [27]:
def get_score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=model(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(),tensor_input.squeeze()).data 
    return math.exp(loss)

In [28]:
def get_score_perplexity(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=model(tensor_input)
    predictions = predictions.squeeze()
    predictions = torch.softmax(predictions, dim=1)
    tensor_input = tensor_input.squeeze()
    log_sum=0
    for idx in range(len(tensor_input)):
        log_sum += torch.log(predictions[idx, tensor_input[idx]])
    
    return math.exp(-log_sum/len(tensor_input))

- The examples below are cherry-picked results.

In [29]:
get_score('안녕하세요. 오늘 아침은 미세먼지가 많이 줄어 들어 맑은 하늘을 볼 수 있었습니다.')

2.087662715284083

### SKT-Brain KoBERT

In [30]:
from tokenization_kobert import KoBertTokenizer
from kobert_transformers import get_kobert_lm

In [31]:
kobert_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
kobert_lm_model = get_kobert_lm()

In [32]:
text = "안녕하세요. SK텔레콤에 가서 핸드폰을 개통할까요?"
tokenized_text = kobert_tokenizer.tokenize(text)
print(tokenized_text)

['▁안', '녕', '하세요', '.', '▁SK', '텔레콤', '에', '▁', '가', '서', '▁', '핸드', '폰', '을', '▁개통', '할', '까', '요', '?']


In [33]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
target = '▁개통'
masked_index = tokenized_text.index(target)
tokenized_text[masked_index] = '[MASK]'

In [34]:
print("Original:", text)
print("Masked:", " ".join(tokenized_text))

Original: 안녕하세요. SK텔레콤에 가서 핸드폰을 개통할까요?
Masked: ▁안 녕 하세요 . ▁SK 텔레콤 에 ▁ 가 서 ▁ 핸드 폰 을 [MASK] 할 까 요 ?


In [35]:
indexed_tokens = kobert_tokenizer.convert_tokens_to_ids(tokenized_text)

# Define sentence A and B indices associated to 1st and 2nd sentences
eos_idx = eos_index(tokenized_text)
# this is for the first sentence. 
segments_ids = [1] * len(tokenized_text)
for i in range(eos_idx):
    segments_ids[i] =0 

In [36]:
# Define sentence A and B indices associated to 1st and 2nd sentences
mask_ids = [1] * len(tokenized_text)
# this is for the first sentence. 
mask_ids[masked_index] = 0

In [37]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
masks_tensors = torch.tensor([mask_ids])

In [38]:
predictions = kobert_lm_model(tokens_tensor, masks_tensors, segments_tensors)

In [39]:
prob_masked_token = predictions[0][0][masked_index]
prob_masked_token.shape

torch.Size([8002])

In [40]:
values, indices = torch.topk(prob_masked_token, 10)

print(values)
print(indices)

tensor([10.1269,  8.3480,  7.9797,  7.6916,  7.6222,  6.8082,  6.5569,  6.5015,
         6.4961,  6.3186], grad_fn=<TopkBackward>)
tensor([2613, 1124, 1119,  856, 4758, 3334, 1111, 3219, 3726,  898])


In [41]:
predicted_token = kobert_tokenizer.convert_ids_to_tokens(indices.numpy())
print(predicted_token)

['▁사용', '▁구입', '▁구매', '▁개통', '▁통화', '▁연결', '▁교체', '▁얘기', '▁이용', '▁검색']


In [42]:
def kobert_generate_topk_words(test_str):
    text = test_str
    tokenized_text = kobert_tokenizer.tokenize(text)
    
    masked_index = tokenized_text.index('▁*')
    tokenized_text[masked_index] = '[MASK]'
    # Convert token to vocabulary indices
    indexed_tokens = kobert_tokenizer.convert_tokens_to_ids(tokenized_text)
    
    eos_idx = eos_index(tokenized_text)

    # Define sentence A and B indices associated to 1st and 2nd sentences
    segments_ids = [1] * len(tokenized_text)
    for i in range(eos_idx):
        segments_ids[i] =0 
        
    mask_ids = [1] * len(tokenized_text)
    mask_ids[masked_index] = 0
    
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    masks_tensors = torch.tensor([mask_ids])
    
    # Predict all tokens
    predictions = kobert_lm_model(tokens_tensor, masks_tensors, segments_tensors)
    prob_masked_token = predictions[0][0][masked_index]

    print("Original:", text)
    print("Masked:", " ".join(tokenized_text))
    
    values, indices = torch.topk(prob_masked_token, 10)
    predicted_token = kobert_tokenizer.convert_ids_to_tokens(indices.numpy())
    print('Predicted top-k tokens: ',  predicted_token)

In [43]:
kobert_generate_topk_words('안녕하세요. 오늘 아침에 아주 맛있는 *을 먹었습니다.')

Original: 안녕하세요. 오늘 아침에 아주 맛있는 *을 먹었습니다.
Masked: ▁안 녕 하세요 . ▁오늘 ▁아침 에 ▁아주 ▁맛 있는 [MASK] 을 ▁먹 었습니다 .
Predicted top-k tokens:  ['▁저녁', '▁밥', '▁음식', '▁아침', '▁죽', '것', '▁약', '▁술', '밥', '▁밤']


In [44]:
kobert_generate_topk_words('안녕하세요. 오늘 *에 아주 맛있는 라면을 먹었습니다.')

Original: 안녕하세요. 오늘 *에 아주 맛있는 라면을 먹었습니다.
Masked: ▁안 녕 하세요 . ▁오늘 [MASK] 에 ▁아주 ▁맛 있는 ▁ 라면 을 ▁먹 었습니다 .
Predicted top-k tokens:  ['▁아침', '▁저녁', '밤', '▁낮', '▁밤', '만', '▁새벽', '▁오후', '▁오전', '▁처음']


In [45]:
kobert_generate_topk_words('안녕하세요. 오늘 아침에 재미있는 *을 읽었습니다.')

Original: 안녕하세요. 오늘 아침에 재미있는 *을 읽었습니다.
Masked: ▁안 녕 하세요 . ▁오늘 ▁아침 에 ▁재미있 는 [MASK] 을 ▁읽 었습니다 .
Predicted top-k tokens:  ['▁책', '▁소설', '▁신문', '글', '신문', '▁그림', '▁글', '▁시', '▁작품', '책']


In [46]:
kobert_generate_topk_words('안녕하세요. 저는 *를 할 때 이 세상에서 제일 행복 합니다.')

Original: 안녕하세요. 저는 *를 할 때 이 세상에서 제일 행복 합니다.
Masked: ▁안 녕 하세요 . ▁저 는 [MASK] 를 ▁할 ▁때 ▁이 ▁세상 에서 ▁제 일 ▁행복 ▁합니다 .
Predicted top-k tokens:  ['▁정치', '▁연애', '▁야구', '▁공부', '▁축구', '▁요리', '▁노래', '▁영화', '▁투표', '▁인터뷰']


In [47]:
def kobert_get_score(sentence):
    tokenize_input = kobert_tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([kobert_tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=kobert_lm_model(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions[0][0],tensor_input.squeeze()).data 
    return math.exp(loss)

In [48]:
kobert_get_score('안녕하세요. 책상 위에 책이 있어서 읽어 보았습니다.')

1.38208281770758

In [49]:
kobert_get_score('안녕하세요. 책상 위에 챙이 있어서 읽어 보았습니다.')

1.3895272035424144

In [50]:
kobert_get_score('안녕하세요. 아침으로 토스트를 먹었다.')

1.6909254883213238

In [51]:
kobert_get_score('안녕하세요. 아침으로 트스트를 먹었다.')

2.093944511886682

In [52]:
kobert_get_score('안녕하세요. 동물원에서 코끼리를 봤습니다.')

1.4528209396508616

In [53]:
kobert_get_score('안녕하세요. 동물원에서 전시회를 봤습니다.')

1.763551134568271

In [54]:
kobert_get_score('반가워. 아버지가 옷을 입으셨다.')

2.714837313945098

In [55]:
kobert_get_score('반가워. 아버지가 옷를 입으셨다.')

4.007834004967372

In [56]:
kobert_get_score('반가워. 아버지가 구두를 입으셨다.')

2.579315082849272